Demo of Task1
MNIST classification with Random Forest, Feed-Forward NN, and CNN.  
We compare preprocessing, training, accuracy, and check edge cases.

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from classifiers.mnist_classifier import MnistClassifier


Download MNIST dataset and normalize data

In [24]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print("Shapes before preprocessing:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

X_train = X_train / 255.0
X_test = X_test / 255.0

Shapes before preprocessing:
X_train: (60000, 28, 28) y_train: (60000,)
X_test: (10000, 28, 28) y_test: (10000,)


Preprocessing
- Random Forest → flatten to `(n_samples, 784)`  
- CNN → add channel dimension `(n_samples, 28, 28, 1)`

Flattened data for Random Forest

In [25]:
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

Add new channel for CNN

In [26]:
X_train_cnn = X_train[..., np.newaxis]
X_test_cnn = X_test[..., np.newaxis]

Random Forest

In [27]:
rf_classifier = MnistClassifier('rf')

rf_classifier.train(X_train_flat, y_train)

y_pred_rf = rf_classifier.predict(X_test_flat)

print('RF Accuracy', accuracy_score(y_test, y_pred_rf))

RF Accuracy 0.9695


Feed-Forward Neural Network

In [28]:
nn_classifier = MnistClassifier('nn')

nn_classifier.train(X_train, y_train)

y_pred_nn = nn_classifier.predict(X_test)

print('NN Accuracy', accuracy_score(y_test, y_pred_nn))

c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9038 - loss: 0.3349
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9509 - loss: 0.1640
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9632 - loss: 0.1221
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9700 - loss: 0.0991
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9746 - loss: 0.0839
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
NN Accuracy 0.9738


Convolutional Neural Network

In [29]:
cnn_classifier = MnistClassifier('cnn')

cnn_classifier.train(X_train_cnn, y_train, epochs=3, batch_size=128)

y_pred_cnn = cnn_classifier.predict(X_test_cnn)

print('CNN Accuracy', accuracy_score(y_test, y_pred_cnn))

c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.9330 - loss: 0.2320
Epoch 2/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.9812 - loss: 0.0618
Epoch 3/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.9868 - loss: 0.0435
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
CNN Accuracy 0.9867


Edge cases
We check unusual inputs:  
- Random Forest: single sample, empty input  
- Feed-Forward NN: single sample, wrong shape  
- CNN: single sample, very small batch

Random Forest: One example

In [30]:
pred_one = rf_classifier.predict(X_test_flat[0:1])
print("Single sample prediction:", pred_one)

Single sample prediction: [7]


Random Forest: Empty array

In [31]:
try:
    pred_empty = rf_classifier.predict(np.array([]).reshape(0, 28*28))
except Exception as e:
    print("Empty input prediction error:", e)

Empty input prediction error: Found array with 0 sample(s) (shape=(0, 784)) while a minimum of 1 is required by RandomForestClassifier.


Feed-Forward Neural Network: One example

In [32]:
pred_one = nn_classifier.predict(X_test[0:1])
print("Single sample prediction:", pred_one)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Single sample prediction: [7]


Feed-Forward Neural Network: Wrong Shape

In [33]:
try:
    pred_wrong_shape = nn_classifier.predict(np.random.rand(10,30,30))
except Exception as e:
    print("Wrong shape prediction error:", e)

Wrong shape prediction error: Graph execution error:

Detected at node sequential_3_1/dense_6_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Users\Zebra\Desktop\Projects\Task1\.conda\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "c:\Users\Zebra\Desktop\Projects\Task1\.cond

Convolutional Neural Network: One example

In [34]:
pred_one = cnn_classifier.predict(X_test_cnn[0:1])
print("Single sample prediction:", pred_one)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Single sample prediction: [7]


Convolutional Neural Network: Very small batch (2 samples)

In [35]:
pred_two = cnn_classifier.predict(X_test_cnn[0:2])
print("Two sample batch prediction:", pred_two)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Two sample batch prediction: [7 2]
